# Using ALOS PALSAR and Forest/Non-Forest Annual Mosaics with the Planetary Computer STAC API

ALOS PALSAR Annual Mosaic consist of radar backscatter in the HH and HV with additional bands for ancillary information about incidince angle, masking, and date of acquisition (per pixel).

In [9]:
from pystac_client import Client

tonga = [-176.2142959,  -22.3487289, -173.7369656,  -15.5661351]
client = Client.open(
    "https://pct-apis-staging.westeurope.cloudapp.azure.com/stac/", #Todo: staging api for now
    ignore_conformance=True,
)
search = client.search(
    collections=["alos-palsar-mosaic"],
    #intersects={"type": "Point", "coordinates": tonga},
)
items = list(search.get_items())
print(f"Returned {len(items)} items")

Returned 500 items


In [10]:
item = items[10]
print(item.assets)

{'HH': <Asset href=https://pceo.blob.core.windows.net/palsar/alos_palsar_mosaic/N01E012_17_sl_HH_F02DAR.tif>,
 'HV': <Asset href=https://pceo.blob.core.windows.net/palsar/alos_palsar_mosaic/N01E012_17_sl_HV_F02DAR.tif>,
 'date': <Asset href=https://pceo.blob.core.windows.net/palsar/alos_palsar_mosaic/N01E012_17_date_F02DAR.tif>,
 'mask': <Asset href=https://pceo.blob.core.windows.net/palsar/alos_palsar_mosaic/N01E012_17_mask_F02DAR.tif>,
 'linci': <Asset href=https://pceo.blob.core.windows.net/palsar/alos_palsar_mosaic/N01E012_17_linci_F02DAR.tif>,
 'tilejson': <Asset href=https://pct-apis-staging.westeurope.cloudapp.azure.com/data/item/tilejson.json?collection=alos-palsar-mosaic&item=N01E012_17_MOS&assets=HH&>,
 'rendered_preview': <Asset href=https://pct-apis-staging.westeurope.cloudapp.azure.com/data/item/preview.png?collection=alos-palsar-mosaic&item=N01E012_17_MOS&assets=HH&>}

In [14]:
import planetary_computer
import xarray
import rioxarray
import xrspatial
from datashader.transfer_functions import shade, stack
from datashader.colors import Elevation

signed_asset = planetary_computer.sign(item.assets["HH"])
data = (
    xarray.open_dataset(signed_asset.href, engine="rasterio")
    .squeeze()
    .drop("band")
    .coarsen({"y": 5, "x": 5})
    .mean()
)

#stack(shade(hillshade, cmap=["white", "gray"]), shade(data, cmap=Elevation, alpha=128))
data

<xarray.Dataset>
Dimensions:      (y: 900, x: 900)
Coordinates:
  * x            (x) float64 12.0 12.0 12.0 12.0 12.01 ... 13.0 13.0 13.0 13.0
  * y            (y) float64 0.9994 0.9983 0.9972 ... 0.001667 0.0005556
    spatial_ref  int64 0
Data variables:
    band_data    (y, x) float32 7.81e+03 7.807e+03 ... 6.9e+03 7.08e+03

## Forest Classification

ALOS Forest/Non-Forest Classification is derived from the ALOS PALSAR Annual Mosaic, and classifies the pixels to detect forest cover.

In [5]:
from pystac_client import Client

tonga = [-176.2142959,  -22.3487289, -173.7369656,  -15.5661351]
client = Client.open(
    "https://pct-apis-staging.westeurope.cloudapp.azure.com/stac/", #Todo: staging api for now
    ignore_conformance=True,
)
search = client.search(
    collections=["alos-fnf-mosaic"],
    #intersects={"type": "Point", "coordinates": tonga},
)
items = list(search.get_items())
print(f"Returned {len(items)} items")

Returned 500 items


In [8]:
item = items[10]
item.assets

{'C': <Asset href=https://pceo.blob.core.windows.net/palsar/alos_fnf_mosaic/N01E012_17_C_F02DAR.tif>,
 'tilejson': <Asset href=https://pct-apis-staging.westeurope.cloudapp.azure.com/data/item/tilejson.json?collection=alos-fnf-mosaic&item=N01E012_17_FNF&assets=C&>,
 'rendered_preview': <Asset href=https://pct-apis-staging.westeurope.cloudapp.azure.com/data/item/preview.png?collection=alos-fnf-mosaic&item=N01E012_17_FNF&assets=C&>}